In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys
import glob
import re
import platform
import importlib
import netCDF4
import multiprocessing as mp
import itertools

base_FP = f'/home/subin/data'
cpuserver_data_FP = f'/home/subin/cpuserver_data'
print(base_FP, cpuserver_data_FP)
sys.path.append(base_FP + '/python_modules')

#hydroAI libs
import HydroAI.HLS as hHLS
import HydroAI.Plot as hPlot
import HydroAI.Data as hData
import HydroAI.Grid as hGrid
importlib.reload(hHLS);
importlib.reload(hPlot);
importlib.reload(hData);
importlib.reload(hGrid);

/home/subin/data /home/subin/cpuserver_data


In [2]:
start_year = 2013; end_year = 2024

temp_path = f'{cpuserver_data_FP}/HLS/Korea/UHI_project_L30/HLS_2013_GJ.nc'
with netCDF4.Dataset(temp_path) as nc_data:
    vars_list = list(nc_data.variables.keys())

print(vars_list)
remove_vars = ['latitude', 'longitude', 'dates']
vars_list = [i for i in vars_list if i not in remove_vars]
print(vars_list)

['latitude', 'longitude', 'dates', 'coastal', 'blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'cirrus', 'tir1', 'tir2', 'qa']
['coastal', 'blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'cirrus', 'tir1', 'tir2', 'qa']


In [3]:
for i, year in enumerate(range(start_year, end_year)):
    
    with netCDF4.Dataset(f'{cpuserver_data_FP}/HLS/Korea/UHI_project_L30/HLS_{year}_GJ.nc') as nc_data:
        nc_lat = nc_data.variables['latitude'][:].data
        nc_lon = nc_data.variables['longitude'][:].data
        nc_dates = nc_data.variables['dates'][:].data

        for date_idx, date in enumerate(nc_dates):
            df_year = pd.DataFrame()
            df_year['latitude'] = nc_lat.flatten()
            df_year['longitude'] = nc_lon.flatten()
            df_year['dates'] = date

            for var_name in vars_list:     
                nc_var = nc_data.variables[var_name][:, :, date_idx].data
                df_year[var_name] = nc_var.flatten()

            if date_idx != 0:
                df_year = pd.concat([df_year_old, df_year], axis=0)

            df_year_old = df_year

    df_year = df_year.dropna(axis=0)

    print(year, np.shape(df_year))
    
    df_year.to_csv(f'{cpuserver_data_FP}/HLS/Korea/UHI_project_L30/HLS_{year}_GJ.csv', sep=',')
    
    del df_year_old
    del df_year

2013 (26811122, 14)
2014 (38688618, 14)
2015 (40473344, 14)
2016 (39499565, 14)
2017 (37928852, 14)
2018 (41973313, 14)
2019 (43778229, 14)
2020 (39936867, 14)
2021 (43516534, 14)
2022 (78018647, 14)
2023 (78528736, 14)


In [ ]:
nc_dates

In [ ]:
#df_year_total.to_csv(f'{cpuserver_data_FP}/HLS/Korea/UHI_project_L30/HLS_2015_2023_GJ.csv', sep=',')